In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import requests
import os

TELEGRAM_TOKEN = "8083975934:AAEFXz0ja40Z470mmZSqGej7g6p80Kt6wh0"  # o ponlo directamente como string
TELEGRAM_CHAT_ID = "1030345806"

def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    payload = {
        "chat_id": TELEGRAM_CHAT_ID,
        "text": message
    }
    try:
        response = requests.post(url, data=payload)
        if response.status_code != 200:
            print("❌ Error en la respuesta de Telegram:")
            print(response.status_code)
            print(response.text)
        else:
            print("✅ Mensaje enviado correctamente.")
    except Exception as e:
        print(f"❌ No se pudo enviar el mensaje de Telegram: {e}")


In [3]:
from financial.momentum.models.kerasAdvanced import KerasAdvancedModelFactory
from financial.momentum.experiment.modelExperiment import ModelExperimentFactory
import os
from dotenv import load_dotenv
from financial.momentum.utilities import find_dotenv
from financial.momentum.utilities import metrics
import financial.data as fd
from financial.io.file.cache import FileCache
import pandas as pd
from financial.lab.experiment import Experiment
from sklearn.metrics import r2_score
from financial.model import Model

import tensorflow as tf
from keras import backend as K
import gc

def reset_gpu():
    K.clear_session()
    tf.keras.backend.clear_session()
    gc.collect()

2025-08-10 07:12:19.444893: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-10 07:12:19.454561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754802739.465518 2051599 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754802739.468862 2051599 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754802739.477553 2051599 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
load_dotenv(dotenv_path=find_dotenv())

cache_path = os.environ["CACHE"]
model_path = os.environ["MODEL"]
print(model_path)
print(os.environ["DATA"])
datastore = fd.CachedDataStore(path=os.environ["DATA"], cache=FileCache(cache_path=cache_path+"/"))
print(datastore) 

/home/manidmt/TFG/OTRI/models/keras
/home/manidmt/TFG/OTRI/data
CachedDataStore with 960 data sources [cache stats: {'size': 0, 'hit': 0, 'miss': 0, 'write': 0, 'read': 0, 'update': 0}]


In [5]:
start_date = "2021-01-01"
end_date = "2025-06-30"
data = datastore.get_data("AAPL", start_date, end_date)
print(data)

Date
2021-01-04    126.239700
2021-01-05    127.800514
2021-01-06    123.498520
2021-01-07    127.712715
2021-01-08    128.815063
                 ...    
2025-06-24    200.300003
2025-06-25    201.559998
2025-06-26    201.000000
2025-06-27    201.080002
2025-06-30    205.169998
Name: AAPL, Length: 1127, dtype: float64


In [6]:
import os
import pickle

def store_results(ticker: str, model_name: str, predictions: pd.Series):

    base_name = f"model-momentum-{model_name}@pred"

    # Guardar .pkl para Wrapper
    with open(os.path.join(cache_path, base_name), "wb") as f:
        pickle.dump(predictions, f)
    print("Prediciones guardadas pickle")
    # Guardar .csv para inspección y web
    predictions.to_csv(os.path.join(model_path, model_name + "_preds.csv"))
    print("Prediciones guardadas csv")
    

In [7]:
factory = KerasAdvancedModelFactory()

In [13]:
from itertools import product

# Arquitecturas a probar
architectures = ["rnn"]
#architectures = ["rnn","lstm", "cnn", "cnn2d", "transformer"]

# Hiperparámetros a explorar
layer_option = [64, 32]
activation = ["relu"]
batch_size = 32
epochs = 10

def ticker_single_models(ticker):

    data = datastore.get_data(ticker, start_date, end_date)    
    target = data[90 + 20:]
    
    for arch in architectures:
        name = f"{arch}_{ticker}_{end_date[:4]}_multiple"
        path = model_path + "/" + name

        try:
            if Model.from_file(name, path=os.environ["MODEL"]):
                print(f"Modelo {name} ya existe. Saltando...")
                #send_telegram_message(f"Modelo {name} ya existe. Saltando...")
                continue
        except FileNotFoundError:
            print(f"📂 Modelo {name} no existe. Procediendo a entrenar...")
            #send_telegram_message(f"Modelo {name} no existe. Procediendo a entrenar...")

        config = {
            "mode": "global",  
            "datastore": datastore,
            "ticker": [ticker, "^GSPC"],
            "model_factory": factory,
            "name": name,
            "start_year": start_date,
            "end_year": end_date,
            "lookahead": 20,
            "horizon": 90,
            "model_params": {
                "architecture": arch,
                "topology": {
                    "layers": layer_option,
                    "activation": {
                        "hidden": activation,
                        "output": "linear"
                    }
                },
                "optimization": {
                    "optimizer": "adam",
                    "loss": "mean_squared_error",
                    "metrics": ["mae"],
                    "epochs": epochs,
                    "batch_size": batch_size,
                    "validation_split": 0.1
                }
            }
        }

        try:
            print("RUNNING MODEL:")
            experiment = ModelExperimentFactory.create_experiment(config)
            experiment.run()
            print("METRICS:")
            preds = experiment.reconstruct_absolute_predictions_from_relative()
            store_results(ticker, name, preds)
            send_telegram_message("Resultados guardados")
            metrics(experiment, preds, target, model_path)
            send_telegram_message("Metricas guardadas")
            reset_gpu()
        except Exception as e:
            print(f"Falló el modelo {name}: {e}")

In [16]:
ticker_single_models("AAPL")
#ticker_single_models("BLK")
#ticker_single_models("BAM")

📂 Modelo rnn_AAPL_2025_multiple no existe. Procediendo a entrenar...
RUNNING MODEL:
features(change(AAPL,1),change(^GSPC,1),change(AAPL,2),change(^GSPC,2),change(AAPL,3),change(^GSPC,3),change(AAPL,4),change(^GSPC,4),change(AAPL,5),change(^GSPC,5),change(AAPL,6),change(^GSPC,6),change(AAPL,7),change(^GSPC,7),change(AAPL,8),change(^GSPC,8),change(AAPL,9),change(^GSPC,9),change(AAPL,10),change(^GSPC,10),change(AAPL,11),change(^GSPC,11),change(AAPL,12),change(^GSPC,12),change(AAPL,13),change(^GSPC,13),change(AAPL,14),change(^GSPC,14),change(AAPL,15),change(^GSPC,15),change(AAPL,16),change(^GSPC,16),change(AAPL,17),change(^GSPC,17),change(AAPL,18),change(^GSPC,18),change(AAPL,19),change(^GSPC,19),change(AAPL,20),change(^GSPC,20),change(AAPL,21),change(^GSPC,21),change(AAPL,22),change(^GSPC,22),change(AAPL,23),change(^GSPC,23),change(AAPL,24),change(^GSPC,24),change(AAPL,25),change(^GSPC,25),change(AAPL,26),change(^GSPC,26),change(AAPL,27),change(^GSPC,27),change(AAPL,28),change(^GSPC,28),c

KeyboardInterrupt: 

In [57]:
index_tickers = [
    "^GSPC",          # S&P 500
    "^IXIC",          # Nasdaq
    "^IBEX",          # Ibex 35
    "AMZN",
    "TSLA",
    "GOOG",
    "BAM",
    "BLK",
]

In [58]:

for ticker in index_tickers:
    ticker_single_models(ticker)

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (None, 90, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 90, 1)     │          2 │ input_layer_10[0… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 90, 1)     │         15 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_20 (Add)        │ (None, 90, 1)     │          0 │ layer_normalizat… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_31          │ (None, 90, 1)     │          0 │ add_20[0][0]      │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 90, 1)     │          2 │ dropout_31[0][0]  │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_30 (Dense)    │ (None, 90, 64)    │        128 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_31 (Dense)    │ (None, 90, 1)     │         65 │ dense_30[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_21 (Add)        │ (None, 90, 1)     │          0 │ dropout_31[0][0], │
│                     │                   │            │ dense_31[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_32          │ (None, 90, 1)     │          0 │ add_21[0][0]      │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1)         │          0 │ dropout_32[0][0]  │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_32 (Dense)    │ (None, 1)         │          2 │ global_average_p… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 216 (868.00 B)

 Trainable params: 214 (856.00 B)

 Non-trainable params: 0 (0.00 B)

...

 Total params: 216 (868.00 B)

 Trainable params: 214 (856.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

None
Modelo transformer_BLK_2025_single ya existe. Saltando...
